In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/dl_course/lesson-21/data/smile-annotations-final.csv"
df = pd.read_csv(file_path, header=None)

In [ ]:
df.columns = ["id", "text", "label"]

In [ ]:
df1 = df[df['label'] == 'surprise']
df1['text'].values

array(['@britishmuseum turquoise eyes!!!',
       '@britishmuseum a death mask??',
       '@britishmuseum @Moonbootica Wow, he was a very old 32.',
       'OMG! I thought she was asleep upstairs!@3speeders @Tate_StIves',
       '@britishmuseum @thehistoryguy Gosh periscope is definitely on trend. High profile people using it and I have not even tried it yet.',
       '@britishmuseum Omg, he looks totally like Al Pacino! http://t.co/jHgsdUsFCn /@schriftart',
       '@britishmuseum Is he holding an orange? Someone had better watch out!',
       '@britishmuseum So much teeth problems un ancient Egypt!!! 😁',
       "@NationalGallery @Londoncraftweek how weird you should say that when I've been at #barberinstitute and took this pic http://t.co/P1iQcqQ8nY",
       '@britishmuseum Cod piece made of armor?',
       '@britishmuseum hmmmm...is it a teapot?? 😊',
       "@appendixjournal @MartinPribble @FitzMuseum_UK So the Swiss Army knife is really a Roman army knife? I'm going to engrave SPQR o

In [ ]:
df['label'].drop_duplicates()
rm_values = ["nocode", "not-relevant", "disgust|angry", "happy|surprise", "happy|sad", "sad|disgust", "sad|angry", "sad|disgust|angry"]
df = df[~df['label'].isin(rm_values)]
df['label'].value_counts()

,count
label,
happy,1137
angry,57
surprise,35
sad,32
disgust,6


In [ ]:
labels = sorted(df["label"].unique())
itos = {i:v for i,v in enumerate(labels)}
stoi = {v:i for i,v in itos.items()}
stoi

{'angry': 0, 'disgust': 1, 'happy': 2, 'sad': 3, 'surprise': 4}

In [ ]:
import json

with open("label_map.json", "w") as f:
    json.dump(itos, f)

In [ ]:
X = df['text'].to_list()
Y = df["label"].apply(lambda row: stoi[row]).values

In [ ]:
from sklearn.model_selection import train_test_split

Xtr, Xtemp, Ytr, Ytemp = train_test_split(
    X, Y,
    train_size=0.7,
    stratify=Y,
    shuffle=True,
    random_state=42
  )

Xval, Xts, Yval, Yts = train_test_split(
    Xtemp, Ytemp,
    train_size=0.5,
    stratify=Ytemp,
    shuffle=True,
    random_state=42
)


In [ ]:
print(pd.DataFrame(Ytr).value_counts())
print(pd.DataFrame(Yval).value_counts())

0
2    795
0     40
4     25
3     22
1      4
Name: count, dtype: int64
0
2    171
0      8
3      5
4      5
1      1
Name: count, dtype: int64


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(Ytr),
    y=Ytr
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"Calculated Class Weights: {class_weights}")

Calculated Class Weights: tensor([ 4.4300, 44.3000,  0.2229,  8.0545,  7.0880], device='cuda:0')


In [ ]:
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss(weight=class_weights)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-large-uncased",
    num_labels=len(labels)
)
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Xtr_tokenized = tokenizer(Xtr, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
Xval_tokenized = tokenizer(Xval, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
Xts_tokenized = tokenizer(Xts, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, X, Y):
    super().__init__()
    self.X = X
    self.Y = Y

  def __len__(self):
    return len(self.X['input_ids'])

  def __getitem__(self, id):
    message = {k: v[id] for k, v in self.X.items()}
    label = torch.tensor(self.Y[id])
    return message, label

Dtr = CustomDataset(Xtr_tokenized, Ytr)
Dval = CustomDataset(Xval_tokenized, Yval)
Dts = CustomDataset(Xts_tokenized, Yts)

In [ ]:
DLtr = DataLoader(Dtr, batch_size=16, shuffle=True)
DLval = DataLoader(Dval, batch_size=16, shuffle=True)
DLts = DataLoader(Dts, batch_size=16, shuffle=False)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
def train_epoch(model, dataloader, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in dataloader:

        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs)
        logits = outputs.logits
        loss = criterion(logits, labels)
        preds = torch.argmax(logits, dim=1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += loss.item()

    acc = correct / total
    return total_loss / len(dataloader), acc

def eval_epoch(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs)
            logits = outputs.logits
            loss = criterion(logits, labels)

            preds = torch.argmax(logits, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    acc = correct / total
    return total_loss / len(dataloader), acc

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, DLtr, criterion)
    val_loss, val_acc = eval_epoch(model, DLval, criterion)

    print(f"""
    Epoch {epoch+1}
    Train loss: {train_loss:.4f}
    Train acc:  {train_acc:.4f}
    Val loss:   {val_loss:.4f}
    Val acc:    {val_acc:.4f}
    """)


    Epoch 1
    Train loss: 1.5050
    Train acc:  0.6907
    Val loss:   1.3737
    Val acc:    0.7842
    

    Epoch 2
    Train loss: 1.3430
    Train acc:  0.7833
    Val loss:   1.3406
    Val acc:    0.9158
    

    Epoch 3
    Train loss: 1.1163
    Train acc:  0.8894
    Val loss:   1.0693
    Val acc:    0.9316
    

    Epoch 4
    Train loss: 0.8427
    Train acc:  0.9368
    Val loss:   0.8804
    Val acc:    0.7895
    

    Epoch 5
    Train loss: 0.5357
    Train acc:  0.9052
    Val loss:   0.9293
    Val acc:    0.8789
    

    Epoch 6
    Train loss: 0.3785
    Train acc:  0.9515
    Val loss:   1.1809
    Val acc:    0.9421
    

    Epoch 7
    Train loss: 0.2648
    Train acc:  0.9345
    Val loss:   0.9656
    Val acc:    0.9263
    

    Epoch 8
    Train loss: 0.1003
    Train acc:  0.9853
    Val loss:   0.8745
    Val acc:    0.8947
    

    Epoch 9
    Train loss: 0.0423
    Train acc:  0.9910
    Val loss:   1.2573
    Val acc:    0.9368
    

    Epoch

In [ ]:
def predict_text(text):
  model.eval()
  inputs = tokenizer(text, return_tensors='pt', padding=True)
  inputs = {k:v.to(device) for k,v in inputs.items()}


  with torch.no_grad():
    out = model(**inputs)
  logits = out.logits
  softmax_probs = torch.softmax(logits, dim=1)
  pred = torch.argmax(logits, dim=1)
  predicted_label = itos[pred.item()]
  return predicted_label

In [ ]:
predict_text("This man lose his keys. He is so upset")

'sad'

In [ ]:
torch.save(model.state_dict(), "emotional_model.pt")